In [467]:
import configparser
from pathlib import Path
import collections
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2.extras import execute_values
from psycopg2 import connect
import numpy
import matplotlib as mpl
from IPython.display import HTML, display
import rick
import numpy as np
import matplotlib.pyplot as plt
from setuptools import setup, find_packages
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [518]:
merge_set = pd.read_sql('''select * from congestion.route_nextbest_ordered 
limit 10              
                          ''',con)

In [522]:
unique_set = merge_set[['start_vid', 'end_vid']].drop_duplicates().iloc[0]
prepare_set = merge_set[(merge_set['start_vid'] == start_vid) & (merge_set['end_vid'] == end_vid)]
print(prepare_set)

   start_vid   end_vid  rank  segment_id  \
0   30414921  30414866     1        2067   
1   30414921  30414866     2        2067   

                                           link_set     length  \
0  [8374555211, 8264120121, 8264397781, 8264120091]  28.079778   
1  [8374555211, 8264120121, 8264397781, 8264120091]  28.079778   

                                                geom  pot_seg  pot_start_vid  \
0  0102000020E61000000500000082E2C798BBD853C048F9...     2072       30414923   
1  0102000020E61000000500000082E2C798BBD853C048F9...     2038       30414866   

   pot_end_vid                                       pot_link_set  pot_length  \
0     30414921                           [8287056821, 8287056811]  130.298716   
1     30414610  [295889901, 295889321, 295890151, 295889691, 2...  450.005186   

                                            pot_geom  ang_diff     sum  
0  0102000020E610000003000000BF0E9C33A2D853C0DE93...  0.873724  7308.0  
1  0102000020E610000009000000CDAFE600

In [469]:
def check_usage(link_set, link_used):
    result = []
    if link_used == []:
        return False
    for i in link_used:
        if collections.Counter(link_set) == collections.Counter(i):
            result.append(True)
        else:
            result.append(False)
    if True in result:
        return True
    else:
        return False

In [516]:
def make_selection(segment_set, min_angle, angle_diff, length_tolerance):
    ## find out number of ranks
    if pd.isna(segment_set['ang_diff'].iloc[0]):
        rank = 0
    else:
        if segment_set['ang_diff'].iloc[0] < min_angle:
            if segment_set.shape[0] == 2:
                rank1=segment_set[(segment_set['rank']==1)]
                rank2=segment_set[(segment_set['rank']==2)]

                if rank2['ang_diff'].iloc[0] - rank1['ang_diff'].iloc[0] < angle_diff and rank2['pot_length'].iloc[0] - rank1['pot_length'].iloc[0] < length_tolerance:
                    rank = 2
                else:
                    rank = 1
            elif segment_set.shape[0] == 1:
                    rank = 1
        else:
            rank =0
       
    return rank

In [553]:
def return_chosen_segments(segment_set, rank):
    if rank != 0:
        whole_set = segment_set[(segment_set['rank']==rank)].iloc[0]
        original_set = whole_set[['start_vid', 'end_vid', 'segment_id', 'link_set', 'length']]
        additional_set = whole_set[['pot_start_vid', 'pot_end_vid', 'pot_seg', 'pot_link_set', 'pot_length']]
        # find which node to use for new start and end_vid
        if original_set['start_vid'] == additional_set['pot_end_vid']:
            new_start_vid = additional_set['pot_start_vid']
            new_end_vid = original_set['end_vid']
        else:
            new_start_vid = original_set['start_vid']
            new_end_vid = additional_set['pot_end_vid'] 
        new_length = additional_set['pot_length'] + original_set['length']
        new_link_set = additional_set['pot_link_set'] + original_set['link_set']
        additional_links = additional_set['pot_link_set']
    else:
        new_start_vid = original_set['start_vid']
        new_end_vid = original_set['end_vid']
        new_length = original_set['length']
        new_link_set = original_set['link_set']
        additional_links = []
    return new_start_vid, new_end_vid, new_length, new_link_set, additional_links

In [556]:
rank = make_selection(prepare_set,25, 5, 50)
new_start_vid, new_end_vid, new_length, new_link_set, additional_links = return_chosen_segments(prepare_set, rank)


30414923


In [475]:
link_used = []
new_sets = []
rows = []
unique_set = merge_set[['start_vid', 'end_vid']].drop_duplicates()
merge_num = 0
for index, key in unique_set.reset_index(drop=True).iterrows(): 
    
    start_vid = unique_set['start_vid'].iloc[index]
    end_vid = unique_set['end_vid'].iloc[index]
    prepare_set = merge_set[(merge_set['start_vid'] == start_vid) & (merge_set['end_vid'] == end_vid)]

    if prepare_set['length'] < 100:
        rank = make_selection(prepare_set, 25,5,50)
        new_start_vid, new_end_vid, new_length, new_link_set, additional_links = return_chosen_segments(prepare_set, rank)
        link_used.append(additional_links)
        link_used.append(new_link_set)
        
        new_sets.append(new_set)

30414921 30414866
[8374555211, 8264120121, 8264397781, 8264120091]


KeyError: "None of [Index(['pot_seg, pot_start_vid, pot_end_vid, pot_length, ang_diff'], dtype='object')] are in the [columns]"

In [470]:
link_used = []
new_sets = []
rows = []
unique_set = merge_set[['start_vid', 'end_vid']].drop_duplicates()
merge_num = 0
# loop through unique sets of intersection
# !!this will be ordered by priority and length eventually!!
for index, key in unique_set.reset_index(drop=True).iterrows(): 
    
    start_vid = unique_set['start_vid'].iloc[index]
    end_vid = unique_set['end_vid'].iloc[index]
    #print(start_vid, end_vid)
    prepare_set = merge_set[(merge_set['start_vid'] == start_vid) & (merge_set['end_vid'] == end_vid)]
    link_set = prepare_set['link_set'].iloc[0]
    
    # Check if this link_set was used to match another set before
    # if not, selection best and merge
    if check_usage(link_set, link_used) == False:
        # if this intersection has an next best one: 
        if pd.isna(prepare_set['new_start_vid'].iloc[0]) == False:
            
            # if there are more than one rank and length is less than 100 or something         
            if prepare_set['length'].iloc[0] < 100:    
                # if there are two possibility for this intersection set
                # then compare which one is better
                if prepare_set['rank'].shape[0] == 2: 
                    rank1=prepare_set[(merge_set['rank']==1)]
                    rank2=prepare_set[(merge_set['rank']==2)]
                    print(prepare_set['rank'].shape[0])
                    # find hausdorffdistance difference and length difference
                    hausorffdiff = abs(rank1['st_hausdorffdistance'].iloc[0]/rank2['st_hausdorffdistance'].iloc[0])
                    lengthdiff = rank1['new_length'].iloc[0] - rank2['new_length'].iloc[0]

                    # make selection based on hausdorffdistance difference and length difference
                    # could try and play with the params a little bit
                    if hausorffdiff < 5 and lengthdiff > 20:
                        select_set = 2
                    else:
                        select_set = 1

                    selection = merge_set[(merge_set['rank']==select_set)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]

                    # find links that were used to merge, append it to another list
                    # also have to find out if it was using a set of link that was used before
                    old_links = selection['link_set'].iloc[0]
                    new_links = selection['new_link_set'].iloc[0]
                    additional_links = list(set(new_links).difference(old_links))

                    if check_usage(additional_links,link_used) == False:
                        new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                        link_used.append(additional_links)
                        link_used.append(old_links)
                        new_sets.append(new_set)
                        merge_num = merge_num + 1
                    else:
                        new_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                        link_used.append(old_links)
                        new_sets.append(new_set)

                # if there is only one possibility
                # see if that possibility is viable (this has not been added)

                elif prepare_set['rank'].shape[0] == 1:
                    # if st_hausdorffdistance < maybe like 50 or something????: then use it, else append
                    selection = merge_set[(merge_set['rank']==1)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                    old_links = selection['link_set'].iloc[0]
                    new_links = selection['new_link_set'].iloc[0]
                    additional_links = list(set(new_links).difference(old_links))

                    if check_usage(additional_links, link_used) == False and selection['st_hausdorffdistance'] < 100:
                        new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                        link_used.append(old_links)
                        link_used.append(additional_links)
                        new_sets.append(new_set)
                        merge_num = merge_num + 1
                    else:
                        new_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                        print('was used before')
                        link_used.append(old_links)
                        new_sets.append(new_set)

            # if the set length is longer than 100 then 
            # check to see if it was used to merge 
            # if yes dont append, if not append 
            else: 
                print('longer than 100m')
                old_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                new_sets.append(old_set)
                link_used.append(link_set)
                
        # if it was not used and does not have next best, append it 
        else:
            print('does not have next best')
            old_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
            new_sets.append(old_set)
            link_used.append(link_set)
    # if link_set as used, continue and dont append
    else:
        print('used')

for i in range(len(new_sets)):
    item = new_sets[i]
    length = item[2].astype(float)
    row = (item[0].astype(float), item[1].astype(float), length, item[3])
    rows.append(row)
        
#sql = '''insert into congestion.test_shortseg(start_vid, end_vid, length, link_set) VALUES %s'''    
#with con:
#    with con.cursor() as cur:
#        execute_values(cur, sql, rows)    

        

KeyError: 'new_start_vid'